In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('camera open failed')
    sys.exit()
    
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)*0.8)
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    
# out = cv2.VideoWriter('output.avi', fourcc,fps, (w,h)) 
   
srcR = cv2.imread('./samples/640x480-blizzard-blue-solid-color-background.jpg')
srcL = cv2.imread('./samples/640x480-dark-gray-solid-color-background.jpg')
srcU = cv2.imread('./samples/640x480-dark-pastel-purple-solid-color-background.jpg')
srcD = cv2.imread('./samples/640x480-deep-sky-blue-solid-color-background.jpg')
dst = cv2.imread('./samples/640x480-magnolia-solid-color-background.jpg')

cv2.namedWindow('dst', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('dst', cv2.WND_PROP_FULLSCREEN,
         cv2.WINDOW_FULLSCREEN)

while True:    
    ret, frame = cap.read()
    
    model = './opencv_face_detector_uint8.pb'
    config = './opencv_face_detector.pbtxt.txt'
    face_net = cv2.dnn.readNet(model, config)

    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123), swapRB = False)
    face_net.setInput(blob)
    out = face_net.forward()

    detect = out[0, 0, :, :]


    for i in range(detect.shape[0]):
        confidence = detect[i, 2]

        if confidence > 0.5:

            x1 = int(detect[i, 3]*w)
            y1 = int(detect[i, 4]*h)
            x2 = int(detect[i, 5]*w)
            y2 = int(detect[i, 6]*h)
            
            center_boxx = ((x2-x1)//2)+x1
            center_boxy = ((y2-y1)//2)+y1
            centerx = w//2
            centery = h//2

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            text = 'Face: {}%'.format(round(confidence*100, 2))
            cv2.putText(frame, text, (x1, y1-1), cv2.FONT_HERSHEY_COMPLEX,
                       1, (0, 0, 255), 1, cv2.LINE_AA)
            if x1 == x1+1:
                cv2.imshow('image', img)
            
            if center_boxx <= centerx and center_boxy <= centery:
                l = abs(centerx - center_boxx)
                d = abs(center_boxy - centery)
            if center_boxx >= centerx and center_boxy <= centery:
                r = abs(center_boxx -centerx)
                d = abs(center_boxy - centery)
            if center_boxx <= centerx and center_boxy >= centery:
                l = abs(centerx - center_boxx)
                u = abs(centery - center_boxy)
            if center_boxx >= centerx and center_boxy >= centery:
                r = abs(center_boxx -centerx)
                u = abs(centery - center_boxy)


            ptsL1 = np.float32([[0,0], [0,480], [640, 0], [640,480]])
            ptsL2 = np.float32([[0,0], [0,480], [200+(l*(-1))+(r*1), 180+(u*(-1))+(d*1)], 
                                [200+(l*(-1))+(r*1),300+(u*(-1))+(d*1)]])

            ptsR1 = np.float32([[0,0], [0,480], [640, 0], [640,480]])
            ptsR2 = np.float32([[440+(l*(-1))+(r*1),180+(u*(-1))+(d*1)], 
                                [440+(l*(-1))+(r*1),300+(u*(-1))+(d*1)], [640, 0], [640,480]])

            ptsU1 = np.float32([[0,0], [0,480], [640, 0], [640,480]])
            ptsU2 = np.float32([[0,0], [200+(l*(-1))+(r*1),180+(u*(-1))+(d*1)], 
                                [640, 0], [440+(l*(-1))+(r*1),180+(u*(-1))+(d*1)]])

            ptsD1 = np.float32([[0,0], [0,480], [640, 0], [640,480]])
            ptsD2 = np.float32([[200+(l*(-1))+(r*1),300+(u*(-1))+(d*1)], [0,480], 
                                [440+(l*(-1))+(r*1), 300+(u*(-1))+(d*1)], [640,480]])

            mtrxR = cv2.getPerspectiveTransform(ptsR1, ptsR2)
            mtrxL = cv2.getPerspectiveTransform(ptsL1, ptsL2)
            mtrxU = cv2.getPerspectiveTransform(ptsU1, ptsU2)
            mtrxD = cv2.getPerspectiveTransform(ptsD1, ptsD2)

            srcR1 = cv2.warpPerspective(srcR, mtrxR, (640, 480))
            srcL1 = cv2.warpPerspective(srcL, mtrxL, (640, 480))
            srcU1 = cv2.warpPerspective(srcU, mtrxU, (640, 480))
            srcD1 = cv2.warpPerspective(srcD, mtrxD, (640, 480))

            maskR = cv2.inRange(srcR1, (0, 0, 50), (255, 255, 255))
            maskL = cv2.inRange(srcL1, (0, 50, 50), (255, 255, 255))
            maskU = cv2.inRange(srcU1, (50, 0, 50), (255, 255, 255))
            maskD = cv2.inRange(srcD1, (50, 50, 0), (255, 255, 255))

            k = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))

            maskR = cv2.erode(maskR, k)
            maskR = cv2.dilate(maskR, k)

            maskL = cv2.erode(maskL, k)
            maskL = cv2.dilate(maskL, k)

            maskU = cv2.erode(maskU, k)
            maskU = cv2.dilate(maskU, k)

            maskD = cv2.erode(maskD, k)
            maskD = cv2.dilate(maskD, k)    

            dst1 = dst.copy()


            cv2.copyTo(srcR1, maskR, dst1)
            cv2.copyTo(srcL1, maskL, dst1)
            cv2.copyTo(srcU1, maskU, dst1)
            cv2.copyTo(srcD1, maskD, dst1)     

            cv2.imshow('dst', dst1)

            
#             cv2.circle(frame, (((x2-x1)//2)+x1,((y2-y1)//2)+y1), 10, (255,0,0), -1)
                    
    
    if not ret:
        print('frame open failed')
        break
    

    
    cv2.imshow('image', frame)
    
    if cv2.waitKey(20) == 27:
        break


        
cap.release()
cv2.destroyAllWindows()

NameError: name 'l' is not defined